<a href="https://colab.research.google.com/github/AyeshaAnzerBCIT/Multisource/blob/main/EEGFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Process .mat files from GCS, store features in GCS, and resume on crash.

- Reads .mat files from:
  gs://eegchild/EEG_extracted/**/mat_format/*.mat
- For each file, loads EEG data, applies bandpass filtering and normalization,
  computes Hjorth parameters and spectral entropy, then saves a CSV to:
  gs://eegchild/processed_features/{subject_id}_features.csv.
- If a CSV for that subject already exists, the subject is skipped (resume functionality).
"""

import os
import gcsfs
import torch
import h5py
import numpy as np
import scipy.signal
import gc
import pandas as pd
from tqdm import tqdm
from scipy.signal import welch
from scipy.stats import entropy

# -------------------------------
# CONFIGURATION
# -------------------------------
KEY_PATH = "Key.json"       # Path to your service account JSON key
BUCKET_NAME = "eegchild"                # Your GCS bucket name
# Use a recursive glob pattern to match files at any depth under EEG_extracted that have a mat_format folder.
MAT_PATTERN = f"gs://{BUCKET_NAME}/EEG_extracted/**/mat_format/*.mat"
OUTPUT_FOLDER = f"gs://{BUCKET_NAME}/processed_features"  # Where feature CSVs will be saved

# Device: use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Create a GCS filesystem object using your service account key.
fs = gcsfs.GCSFileSystem(token=KEY_PATH)

# -------------------------------
# HELPER FUNCTIONS
# -------------------------------
def list_mat_files():
    """
    List all .mat files in GCS matching the pattern.
    """
    mat_files = fs.glob(MAT_PATTERN)
    print(f"Found {len(mat_files)} .mat files in GCS using pattern:\n  {MAT_PATTERN}")
    return mat_files

def subject_features_exist(subject_id):
    """
    Check if the features CSV for this subject already exists in GCS.
    """
    out_path = f"{OUTPUT_FOLDER}/{subject_id}_features.csv"
    return fs.exists(out_path)

def save_features_to_gcs(subject_id, df_features):
    """
    Save the DataFrame of features as a CSV directly to GCS.
    """
    out_path = f"{OUTPUT_FOLDER}/{subject_id}_features.csv"
    with fs.open(out_path, 'w') as f:
        df_features.to_csv(f, index=False)
    print(f"Saved features for {subject_id} -> {out_path}")

def load_mat_file_gcs(file_path):
    """
    Loads EEG data from a .mat file on GCS using h5py.
    Expects the .mat file to have an 'EEG' group with 'data' (EEG signals)
    and optionally 'srate' (sampling rate).
    """
    try:
        with fs.open(file_path, 'rb') as f:
            with h5py.File(f, 'r') as mat_file:
                data = {}
                if 'EEG' in mat_file:
                    eeg_group = mat_file['EEG']
                    if 'data' in eeg_group:
                        eeg_data = eeg_group['data'][()]
                        eeg_data = torch.tensor(eeg_data, dtype=torch.float32, device=device)
                        # Adjust orientation: if shape is (channels, time) but data is stored as (time, channels)
                        if eeg_data.shape[1] > eeg_data.shape[0]:
                            eeg_data = eeg_data.T
                        data['EEG'] = eeg_data
                    if 'srate' in eeg_group:
                        srate = eeg_group['srate'][()]
                        if srate.shape == (1, 1):
                            srate = srate[0, 0]
                        data['srate'] = float(srate)
                    else:
                        data['srate'] = 256.0  # default sampling rate
                return data if 'EEG' in data else None
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

def bandpass_filter_torch(eeg_data, lowcut=0.5, highcut=50.0, fs=256.0, order=5):
    """
    Applies a bandpass filter to EEG data using SciPy's filtfilt and returns a torch tensor.
    Expects EEG data of shape [channels, time].
    """
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = scipy.signal.butter(order, [low, high], btype='band')
    eeg_np = eeg_data.cpu().numpy()
    filtered_np = scipy.signal.filtfilt(b, a, eeg_np, axis=1)
    # Use .copy() to remove negative strides.
    return torch.tensor(filtered_np.copy(), dtype=torch.float32, device=device)

def normalize_signal_torch(eeg_data):
    """
    Normalizes EEG signals to the range [-1, 1].
    """
    return ((eeg_data - eeg_data.min()) / (eeg_data.max() - eeg_data.min()) * 2 - 1)

def compute_psd_torch(eeg_data, fs=256):
    """
    Computes the power spectral density (PSD) using Welch's method for each channel.
    """
    eeg_np = eeg_data.cpu().numpy()
    psd_list, freq_list = [], []
    for channel in eeg_np:
        freqs, psd = welch(channel, fs, nperseg=fs)
        psd_list.append(psd)
        freq_list.append(freqs)
    return np.array(psd_list), np.array(freq_list)

def hjorth_parameters(eeg_np):
    """
    Computes Hjorth Mobility and Complexity for each channel.
    """
    variance = np.var(eeg_np, axis=1)
    first_derivative = np.diff(eeg_np, axis=1)
    var_deriv = np.var(first_derivative, axis=1)
    mobility = np.sqrt(var_deriv / variance)
    complexity = np.sqrt(np.var(np.diff(first_derivative, axis=1), axis=1) / var_deriv) / mobility
    return mobility, complexity

def spectral_entropy(eeg_np, fs=256):
    """
    Computes the spectral entropy for each channel.
    """
    psd_values, _ = compute_psd_torch(torch.tensor(eeg_np, dtype=torch.float32), fs=fs)
    psd_norm = psd_values / np.sum(psd_values, axis=1, keepdims=True)
    return np.apply_along_axis(entropy, 1, psd_norm, base=2)

# -------------------------------
# MAIN PIPELINE
# -------------------------------
def process_all_patients():
    mat_files = list_mat_files()
    if not mat_files:
        print("No .mat files found in GCS.")
        return

    for file_path in tqdm(mat_files, desc="Processing .mat files"):
        # Extract subject ID from the filename.
        subject_id = os.path.basename(file_path).replace(".mat", "")

        # Resume: Skip processing if features CSV already exists.
        if subject_features_exist(subject_id):
            print(f"Skipping {subject_id} (features already exist).")
            continue

        # Load EEG data from the .mat file on GCS.
        data = load_mat_file_gcs(file_path)
        if not data:
            continue
        eeg_data = data["EEG"]
        fs_rate = data["srate"]

        # Preprocessing: Apply bandpass filtering and normalization.
        filtered = bandpass_filter_torch(eeg_data, fs=fs_rate)
        normed = normalize_signal_torch(filtered)

        # Feature extraction: Compute Hjorth parameters and spectral entropy.
        normed_np = normed.cpu().numpy()  # Move to CPU for numerical operations
        mobility, complexity = hjorth_parameters(normed_np)
        entropies = spectral_entropy(normed_np, fs=fs_rate)

        # Organize features into a DataFrame.
        df_features = pd.DataFrame({
            "Mobility": mobility,
            "Complexity": complexity,
            "Spectral_Entropy": entropies
        })

        # Save the features CSV directly to GCS.
        save_features_to_gcs(subject_id, df_features)

        # Cleanup to free memory.
        del eeg_data, filtered, normed, normed_np, data
        torch.cuda.empty_cache()
        gc.collect()

if __name__ == "__main__":
    print("Starting GCS-based EEG .mat processing for all patients, with resume-on-crash...\n")
    process_all_patients()
    print("Done!")


Using device: cuda:0
Starting GCS-based EEG .mat processing for all patients, with resume-on-crash...

Found 1489 .mat files in GCS using pattern:
  gs://eegchild/EEG_extracted/**/mat_format/*.mat


Processing .mat files:   0%|                   | 2/1489 [00:00<01:25, 17.33it/s]

Skipping gip_A00051826005 (features already exist).
Skipping gp_A00051826001 (features already exist).
Skipping gp_A00051826002 (features already exist).


Processing .mat files:   0%|                   | 4/1489 [00:00<01:22, 17.93it/s]

Skipping gp_A00051826003 (features already exist).


...

Saved features for oip_A00063377 007 -> gs://eegchild/processed_features/oip_A00063377 007_features.csv


Processing .mat files: 100%|█████████████▉| 1488/1489 [6:18:55<00:52, 52.53s/it]

Saved features for oip_A00063377 008 -> gs://eegchild/processed_features/oip_A00063377 008_features.csv


Processing .mat files: 100%|██████████████| 1489/1489 [6:19:32<00:00, 15.29s/it]

Saved features for oip_A00063377 010 -> gs://eegchild/processed_features/oip_A00063377 010_features.csv
Done!


In [ ]:
import os
import pandas as pd
import gcsfs

# -------------------------------
# CONFIGURATION
# -------------------------------
KEY_PATH = "Key.json"  # Path to your service account JSON key
BUCKET_NAME = "eegchild"
PROCESSED_FEATURES_FOLDER = f"gs://{BUCKET_NAME}/processed_features"  # Folder with feature CSVs

# Create a GCS filesystem object (reuse your KEY_PATH)
fs = gcsfs.GCSFileSystem(token=KEY_PATH)

# -------------------------------
# 1. List all CSV files in the processed features folder
# -------------------------------
csv_pattern = os.path.join(PROCESSED_FEATURES_FOLDER, "*.csv")
csv_files = fs.glob(csv_pattern)
print(f"Found {len(csv_files)} CSV files in {PROCESSED_FEATURES_FOLDER}.")

# -------------------------------
# 2. Load each CSV and add extra columns with error handling
# -------------------------------
dataframes = []
for file_path in csv_files:
    try:
        # Open and read CSV from GCS
        with fs.open(file_path, 'r') as f:
            df = pd.read_csv(f)

        # Check if the DataFrame is empty (no columns)
        if df.empty or df.columns.empty:
            print(f"Skipping empty file: {file_path}")
            continue

        # Get the base file name (e.g., "A123_features.csv")
        file_name = os.path.basename(file_path)
        df['file_name'] = file_name

        # Extract patient_id from the file name.
        # Assumes that the file name starts with the patient id (e.g., "A123_features.csv").
        patient_id = file_name.split("_")[0]
        df['patient_id'] = patient_id

        dataframes.append(df)
        print(f"Loaded file: {file_name}")
    except pd.errors.EmptyDataError:
        print(f"EmptyDataError: No columns to parse from file: {file_path}. Skipping.")
    except Exception as e:
        print(f"Error reading {file_path}: {e}. Skipping.")

# -------------------------------
# 3. Merge all DataFrames into one
# -------------------------------
if dataframes:
    merged_df = pd.concat(dataframes, ignore_index=True)
    print("Merged DataFrame shape:", merged_df.shape)
else:
    print("No valid CSV files found to merge.")
    merged_df = pd.DataFrame()

# -------------------------------
# 4. Save the merged DataFrame to GCS
# -------------------------------
merged_file_path = os.path.join(PROCESSED_FEATURES_FOLDER, "merged_features.csv")
with fs.open(merged_file_path, 'w') as f:
    merged_df.to_csv(f, index=False)

print(f"Merged CSV file saved to: {merged_file_path}")


Found 1492 CSV files in gs://eegchild/processed_features.
Loaded file: behavioral_features.csv
Loaded file: bip_A00053375001_features.csv
Loaded file: bip_A00053375002_features.csv
Loaded file: bip_A00053375003_features.csv
Loaded file: bip_A00053375004_features.csv
Loaded file: bip_A00053375008_features.csv
Loaded file: bip_A00053375009_features.csv
Loaded file: bip_A00053398_part2003_features.csv
Loaded file: bip_A00053398_part2004_features.csv
Loaded file: bip_A00053398_part2005_features.csv
Loaded file: bip_A00053398_part2006_features.csv
Loaded file: bip_A00053909002_features.csv
Loaded file: bip_A00054215001_features.csv
Loaded file: bip_A00054215004_features.csv
Loaded file: bip_A00054215010_features.csv
Loaded file: bip_A00054215011_features.csv
Loaded file: bip_A00054400015_features.csv
Loaded file: bip_A00054597013_features.csv
Loaded file: bip_A00054597014_features.csv
Loaded file: bip_A00054694003_features.csv
Loaded file: bip_A00054766011_features.csv
Loaded file: bip_A000

/tmp/ipykernel_1571142/1146713451.py:30: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


Loaded file: merged_features.csv
Loaded file: oip_A00051826006_features.csv
Loaded file: oip_A00051826007_features.csv
Loaded file: oip_A00051826008_features.csv
Loaded file: oip_A00051826009_features.csv
Loaded file: oip_A00051826010_features.csv
Loaded file: oip_A00051826011_features.csv
Loaded file: oip_A00053375005_features.csv
Loaded file: oip_A00053375007_features.csv
Loaded file: oip_A00053398_part2001_features.csv
Loaded file: oip_A00053398_part2002_features.csv
Loaded file: oip_A00053398_part2010_features.csv
Loaded file: oip_A00053398_part2011_features.csv
Loaded file: oip_A00053440_part2004_features.csv
Loaded file: oip_A00053460003_features.csv
Loaded file: oip_A00053460004_features.csv
Loaded file: oip_A00053460005_features.csv
Loaded file: oip_A00053460006_features.csv
Loaded file: oip_A00053460010_features.csv
Loaded file: oip_A00053480001_features.csv
Loaded file: oip_A00053480002_features.csv
Loaded file: oip_A00053480003_features.csv
Loaded file: oip_A00053480005_feat

In [ ]:
import gcsfs
import pandas as pd
import os

# Set GOOGLE_APPLICATION_CREDENTIALS environment variable
# This assumes your Key.json file is in the current directory
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Key.json"

# Initialize GCS file system without explicitly passing the token
fs = gcsfs.GCSFileSystem()


# Define the GCS file path for the third modality
THIRD_MODALITY_FILE = "eegchild/processed_features/merged_features.csv"

# Initialize GCS file system
fs = gcsfs.GCSFileSystem()

# Load the third modality dataset
try:
    with fs.open(THIRD_MODALITY_FILE, 'r') as f:
        df_third_modality = pd.read_csv(f)
    print(f"Loaded {len(df_third_modality)} rows from {THIRD_MODALITY_FILE}\n")

    # Display basic dataset information
    print("Dataset Info:")
    print(df_third_modality.info())

    # Show the first few rows
    print("\nSample Rows:")
    print(df_third_modality.head())

    # Check for missing values
    print("\nMissing Values:")
    print(df_third_modality.isnull().sum())

    # Summary statistics
    print("\nSummary Statistics:")
    print(df_third_modality.describe())

except Exception as e:
    print(f"Error loading {THIRD_MODALITY_FILE}: {e}")


/tmp/ipykernel_1571142/872443576.py:22: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_third_modality = pd.read_csv(f)


Loaded 435772691 rows from eegchild/processed_features/merged_features.csv

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435772691 entries, 0 to 435772690
Data columns (total 12 columns):
 #   Column            Dtype  
---  ------            -----  
 0   file              object 
 1   mean_rt           float64
 2   std_rt            float64
 3   median_rt         float64
 4   min_rt            float64
 5   max_rt            float64
 6   accuracy          float64
 7   file_name         object 
 8   patient_id        object 
 9   Mobility          float64
 10  Complexity        float64
 11  Spectral_Entropy  float64
dtypes: float64(9), object(3)
memory usage: 39.0+ GB
None

Sample Rows:
                              file  mean_rt  std_rt  median_rt  min_rt  \
0  A00051826_SAIIT_2AFC_Block1.csv     86.0     0.0       86.0    86.0   
1  A00051826_SAIIT_2AFC_Block2.csv     87.0     0.0       87.0    87.0   
2  A00051826_SAIIT_2AFC_Block3.csv     88.0     0.0       88.0   